<a href="https://colab.research.google.com/github/julija-dmrk/data-mining/blob/main/4th_week.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

logistic regression classification

In [74]:
import matplotlib.pyplot as plt   # visualization
import seaborn as sns             # visualization
import numpy as np                # data manipulation
import pandas as pd               # data manipulation, data processing, CSV file I/O (e.g. pd.read_csv
from sklearn import datasets, metrics, pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

# from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, StratifiedKFold, LeaveOneOut, cross_val_score, cross_validate, GridSearchCV, train_test_split
from sklearn.model_selection import cross_val_score, cross_validate 

In [75]:
url = "https://raw.githubusercontent.com/VitaT/ML-python/main/data/suicides.csv"
data = pd.read_csv(url)

In [76]:
# cleaning up
# spliting country-year column
data['year'] = data["country-year"].str[-4:]
data['country'] = data["country-year"].str.split("\d+", expand = True)[0]
# removing ID and country-year columns
data.drop(["ID", "country-year"], axis=1, inplace=True)

In [77]:
# renaming columns for more convenient use
data.rename(columns={'gdp_for_year ($)':'gdp_year', 'gdp_per_capita ($)':'gdp_capita', 'suicides/100k pop':'suicide_norm', "HDI for year":"hdi_year"}, inplace=True)

In [78]:
data.isna().sum()
# suicide per 100k people can be calculated from suicides_no and population. Let's check it they are the same
# We can overwrite suicide_norm
data["suicide_norm"] = round(data["suicides_no"] / data["population"] * 100000,  2)

In [79]:
#Classification
target = data["age"]
data.sex.replace({'male':1,'female':0}, inplace=True)
features = data[["sex", "suicides_no", "year"]]

                          
#features = pd.get_dummies(features, columns=["sex", "year", "generation", "country"])


In [80]:
# Data preparatation. 
# Create standardizer and standardize features
features_standardized = StandardScaler().fit_transform(features)
# use train and test split
# multiple classification, gr = 3
x, x_out, y, y_out = train_test_split(features_standardized, target, test_size = 0.1, random_state = 0)  # for multiple category clasification
# binary classification, gr = 2
x2, x2_out, y2, y2_out = train_test_split(features_standardized[:100,], target[:100], test_size = 0.1, random_state = 0)  # for binary clasification

In [81]:
# Create logistic regression object
log_regr = LogisticRegression(random_state=0)
# Train model
cv_log_reg = cross_validate(log_regr, x2, y2, cv=10, error_score="accuracy", return_train_score=True)

for key in ["test_score", "train_score"]:
  print(key)
  print("Accuracy: %0.2f (+/- %0.2f)" % (cv_log_reg.get(key).mean(), cv_log_reg.get(key).std() * 2))
# 2 SD is roughly 95 % of data.

log_regr.fit(x2, y2) # train

# about model
log_regr.classes_
log_regr.coef_
log_regr.intercept_

log_regr.predict(x2_out)  # predictions
log_regr.predict_proba(x2_out) # prediction probabilities

pred_out = log_regr.predict(x2_out)
pred_train = log_regr.predict(x2)

print('The accuracy of the Logistic Regression is', metrics.accuracy_score(log_regr.predict(x2_out), y2_out))


test_score
Accuracy: 0.04 (+/- 0.11)
train_score
Accuracy: 0.20 (+/- 0.02)
The accuracy of the Logistic Regression is 0.0
